<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Reinforcement Learning for Finance

**Chapter 09 &mdash; Optimal Execution**

&copy; Dr. Yves J. Hilpisch

<a href="https://tpq.io" target="_blank">https://tpq.io</a> | <a href="https://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

### Please use the "Python 3.10, Tensorflow 2.10" kernel.

## Model Implementation

In [ ]:
!git clone https://github.com/tpq-classes/rl_4_finance.git
import sys
sys.path.append('rl_4_finance')


In [ ]:
import math
import random
import numpy as np
import pandas as pd
from pylab import plt, mpl

In [ ]:
plt.style.use('seaborn-v0_8')
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(suppress=True)

In [ ]:
class AlmgrenChriss:
    def __init__(self, T, N, S0, sigma, X, eta, gamma, lamb):
        self.T = T              
        self.N = N           
        self.dt = T / N
        self.S0 = S0
        self.sigma = sigma
        self.X = X
        self.eta = eta
        self.gamma = gamma
        self.lamb = lamb

In [ ]:
class AlmgrenChriss(AlmgrenChriss):
    def optimal_execution(self):
        kappa = np.sqrt(self.lamb * self.sigma ** 2 / self.gamma)
        t = np.linspace(0, self.T, self.N + 1)
        xt_sum = self.X * np.sinh(kappa * (self.T - t)) / np.sinh(kappa * self.T)
        xt = -np.diff(xt_sum, prepend=0)
        xt[0] = 0
        return t, xt

In [ ]:
T = 10  # <1>
N = 10  # <2>
S0 = 1  # <3>
sigma = 0.15  # <4>
X = 1  # <5>
eta = 0.1  # <6>
gamma = 0.1  # <7>
lamb = 3e-1  # <8>

In [ ]:
ac = AlmgrenChriss(T, N, S0, sigma, X, eta, gamma, lamb)

In [ ]:
t, xt = ac.optimal_execution()

In [ ]:
t

In [ ]:
xt.round(2)  # <9>

In [ ]:
xt.sum()

In [ ]:
ac.lamb = 1e-4  # <10>

In [ ]:
t, xt_ = ac.optimal_execution()
xt_.round(2)  # <11>

In [ ]:
plt.plot(t, ac.X - xt.cumsum(), 'r', lw=1,
         label='high $\\lambda$ (position)')
plt.plot(t, xt, 'rs', markersize=4,
         label='high $\\lambda$ (trade)')
plt.plot(t, ac.X- xt_.cumsum(), 'b--', lw=1,
         label='low $\\lambda$ (position)')
plt.plot(t, xt_, 'bo', markersize=4,
         label='low $\\lambda$ (trade)')
plt.legend();

In [ ]:
from numpy.random import default_rng

In [ ]:
class AlmgrenChriss(AlmgrenChriss):
    def simulate_stock_price(self, xt, seed=None):
        rng = default_rng(seed=seed)
        S = np.zeros(self.N + 1)  # <1>
        S[0] = self.S0  # <1>
        P = np.zeros(self.N + 1)  # <2>
        P[0] = self.S0  # <2>
        for t in range(1, self.N + 1):
            dZ = rng.normal(0, np.sqrt(self.dt))
            S[t] = S[t - 1] + sigma * dZ  # <1>
            P[t] = S[t] - self.eta * xt[:t + 1].sum()  # <2>
        return S, P

In [ ]:
ac = AlmgrenChriss(T, N, S0, sigma, X, eta, gamma, lamb)

In [ ]:
t, xt = ac.optimal_execution()

In [ ]:
xt.round(2)

In [ ]:
seed = 250

In [ ]:
S, P = ac.simulate_stock_price(xt, seed=seed)

In [ ]:
ac.lamb = 1e-4

In [ ]:
t, xt_ = ac.optimal_execution()

In [ ]:
xt_.round(2)

In [ ]:
S, P_ = ac.simulate_stock_price(xt_, seed=seed)

In [ ]:
plt.plot(t, S, 'b', lw=1, label='simulated stock price path')
plt.plot(t, P, 'r--', lw=1, label='adjusted path (high $\\lambda$)');
plt.plot(t, P_, 'g:', lw=1, label='adjusted path (low $\\lambda$)')
plt.legend();

In [ ]:
class AlmgrenChriss(AlmgrenChriss):
    def calculate_costs(self, xt):
        temporary_cost = np.sum(self.gamma * (xt / self.dt) ** 2 * self.dt)
        permanent_cost = np.sum(self.eta * np.cumsum(xt) * xt)
        execution_risk = self.lamb * self.sigma ** 2 * np.sum(
            (np.cumsum(xt[::-1])[::-1] / self.dt) ** 2 * self.dt)
        TEC = temporary_cost + permanent_cost + execution_risk
        return temporary_cost, permanent_cost, execution_risk, TEC

In [ ]:
ac = AlmgrenChriss(T, N, S0, sigma, X, eta, gamma, lamb)

In [ ]:
t, xt = ac.optimal_execution()

In [ ]:
tc, pc, er, TEC = ac.calculate_costs(xt)

In [ ]:
print(f'lambda = {ac.lamb}')
print(f'temporary cost = {tc:7.4f}')
print(f'permanent cost = {pc:7.4f}')
print(f'execution risk = {er:7.4f}')
print(f'total ex. cost = {TEC:7.4f}')  # <1>

In [ ]:
ac.lamb = 1e-4

In [ ]:
t, xt_ = ac.optimal_execution()

In [ ]:
tc, pc, er, TEC = ac.calculate_costs(xt_)

In [ ]:
print(f'lambda = {ac.lamb}')
print(f'temporary cost = {tc:7.4f}')
print(f'permanent cost = {pc:7.4f}')
print(f'execution risk = {er:7.4f}')
print(f'total ex. cost = {TEC:7.4f}')  # <2>

## Execution Environment

In [ ]:
class observation_space:
    def __init__(self, n):
        self.shape = (n,)

In [ ]:
class action_space:
    def __init__(self, n):
        self.n = n
    def seed(self, seed):
        random.seed(seed)
    def sample(self):
        return random.random()  # <1>

In [ ]:
class Execution:
    def __init__(self, T, N, S0, sigma, X, eta, gamma, lamb):
        self.T = T              
        self.N = N           
        self.dt = T / N
        self.S0 = S0
        self.sigma = sigma
        self.X = X
        self.eta = eta
        self.gamma = gamma
        self.lamb = lamb
        self.episode = 0
        self.observation_space = observation_space(3)
        self.osn = self.observation_space.shape[0]
        self.action_space = action_space(1)

In [ ]:
class Execution(Execution):
    def _simulate_data(self, seed=None):
        self.dt = T / N
        if hasattr(self, 'seed_'):
            seed = self.seed_
        rng = default_rng(seed=seed)
        self.S = np.zeros(self.N + 1)
        self.S[0] = self.S0
        for t in range(1, self.N + 1):
            dZ = rng.normal(0, np.sqrt(self.dt))
            self.S[t] = self.S[t - 1] + self.sigma * dZ
        return self.S

In [ ]:
class Execution(Execution):
    def _get_state(self):
        # St = self.S[self.bar]
        # Pt = self.S[self.bar] - self.eta * self.xt.sum()
        return np.array([self.X_,
                         self.bar / self.N,
                         self.x]), {}
    def seed(self, seed=None):
        self.seed = seed
    def reset(self):
        self.bar = 0
        self.x = 0
        self.x_ = 0
        self.treward = 0
        self.episode += 1
        self.X_ = self.X
        self.xt = np.zeros(self.N + 1)
        self.tec = pd.DataFrame(
            {'pc': 0, 'tc': 0, 'er': 0}, index=[0])
        self._simulate_data()
        self.state, _ = self._get_state()
        return self.state, _

In [ ]:
class Execution(Execution):
    def step(self, action):
        self.bar += 1
        self.xt[self.bar] = action
        self.x_ = self.x
        self.x = action
        self.X_ -= action
        pc = np.sum(self.eta * np.cumsum(self.xt) * self.xt)
        tc = np.sum(self.gamma * (self.xt / self.dt) ** 2 * self.dt)
        er = self.lamb * self.sigma ** 2 * np.sum(
            (np.cumsum(self.xt[::-1])[::-1] / self.dt) ** 2 * self.dt)
        df = pd.DataFrame({'pc': tc, 'tc': pc, 'er': er}, index=[0])
        self.tec = pd.concat((self.tec, df))
        tec = self.tec.diff().fillna(0).iloc[-1]
        cost = tec.sum()
        self.state, _ = self._get_state()
        pen = 0
        reward = 0
        if self.bar < self.N:
            if self.X_ < 0.05:
                done = False  # if True it might stop early (before N is reached)
            else:
                #pen = abs(self.x - self.x_)
                #reward = self.x
                done = False
        elif self.bar == self.N:
            pen = abs(self.X_)
            done = True
        return self.state, reward-(cost+pen), done, False, {}

In [ ]:
execution = Execution(T, N, S0, sigma, X, eta, gamma, lamb=1e-4)

In [ ]:
execution.reset()
execution.step(1.0)

In [ ]:
execution.reset()

In [ ]:
execution.step(0.5)

In [ ]:
execution.step(0.5)

In [ ]:
execution.reset()
for i in range(10):
    execution.step(0.1)
-execution.tec.diff().sum().sum()

## Random Agent

In [ ]:
execution = Execution(T, N, S0, sigma, X, eta, gamma, lamb=1e-4)

In [ ]:
def gen_rn():
    alpha = np.ones(N)
    rn = np.random.dirichlet(alpha)
    rn = np.insert(rn, 0, 0)
    return rn

In [ ]:
rn = gen_rn()
rn

In [ ]:
rn.sum()

In [ ]:
for _ in range(5):
    execution.reset()
    rn = gen_rn()
    print(rn.round(3))
    for i in range(1, 11):
        execution.step(rn[i])
    print(execution.tec.diff().sum().sum())
    print()

In [ ]:
execution = Execution(T, N, S0, sigma, X, eta, gamma, lamb=lamb)

In [ ]:
for _ in range(5):
    execution.reset()
    rn = gen_rn()
    print(rn.round(3))
    for i in range(1, 11):
        execution.step(rn[i])
    print(execution.tec.iloc[-1].sum())
    print()

## Execution Agent 

**_THIS IS WORK IN PROGRESS._**

In [ ]:
import os
import warnings
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
from dqlagent import *

In [ ]:
# random.seed(100)
# tf.random.set_seed(100)

In [ ]:
opt = keras.optimizers.legacy.Adam
# opt = keras.optimizers.legacy.RMSprop

In [ ]:
class ExecutionAgent(DQLAgent):
    def _create_model(self, hu, lr):
        self.model = Sequential()
        self.model.add(Dense(hu, input_dim=self.n_features,
                        activation='relu'))
        self.model.add(Dense(hu, activation='relu'))
        self.model.add(Dense(1, activation='linear'))
        self.model.compile(loss='mse',
                optimizer=opt(learning_rate=lr))

In [ ]:
from scipy.optimize import minimize

In [ ]:
class ExecutionAgent(ExecutionAgent):
    def opt_action(self, state):
        bnds = [(0, state[0, 0])]
        def f(state, x):
            s = np.copy(state)
            s[0, 2] = x
            return self.model.predict(s)[0, 0]
        action = minimize(lambda x: -f(state, x),
                          state[0, 0] / 2,
                          bounds=bnds,
                          method='Powell'
                         )['x'][0]
        return action
        
    def act(self, state):
        if random.random() <= self.epsilon or self.episodes < 250:
            return min(self.rn[self.f], state[0, 0])
        else: 
            return self.opt_action(state)

In [ ]:
class ExecutionAgent(ExecutionAgent):
    def replay(self):
        batch = random.sample(self.memory, self.batch_size)
        for state, action, next_state, reward, done in batch:
            if not done:
                action = self.opt_action(next_state)
                ns = np.copy(next_state)
                ns[0, 2] = action
                reward = (self.gamma *
                    self.model.predict(ns)[0, 0])
            reward = np.array([reward])
            self.model.fit(state, reward, epochs=1,
                           verbose=False)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
class ExecutionAgent(ExecutionAgent):
    def learn(self, episodes):
        for e in range(1, episodes + 1):
            self.episodes += 1
            state, _ = self.env.reset()
            state = self._reshape(state)
            treward = 0
            alpha = np.ones(self.env.N)
            rn = np.random.dirichlet(alpha)
            self.rn = np.insert(rn, 0, 0)
            for f in range(1, 5000):
                self.f = f
                action = self.act(state)
                next_state, reward, done, trunc, _ = self.env.step(action)
                treward += reward
                next_state = self._reshape(next_state)
                self.memory.append(
                    [state, action, next_state, reward, done])
                state = next_state 
                if done:
                    self.trewards.append(treward)
                    self.max_treward = max(self.max_treward, treward)
                    templ = f'episode={self.episodes:4d} | '
                    templ += f'treward={treward:7.3f}'
                    templ += f' | max={self.max_treward:7.3f}'
                    print(templ, end='\r')
                    break
            if len(self.memory) > self.batch_size:
                self.replay()
        print()

In [ ]:
class ExecutionAgent(ExecutionAgent):
    def test(self, episodes, verbose=True):
        for e in range(1, episodes + 1):
            state, _ = self.env.reset()
            state = self._reshape(state)
            treward = 0
            for _ in range(1, self.env.N + 1):
                action = self.opt_action(state)
                state, reward, done, trunc, _ = self.env.step(action)
                state = self._reshape(state)
                treward += reward
                if done:
                    templ = f'total reward={treward:4.3f}'
                    if verbose:
                        print(templ)
                    break
            print(self.env.xt)

In [ ]:
execution = Execution(T, N, S0, sigma, X, eta, gamma, lamb=1e-4)

In [ ]:
executionagent = ExecutionAgent(None, feature=None, n_features=3,
                     env=execution, hu=24, lr=0.0001)

In [ ]:
executionagent.gamma = 0.99

In [ ]:
executionagent.epsilon_decay = 0.999

In [ ]:
episodes = 100

In [ ]:
%time executionagent.learn(episodes)

In [ ]:
execution.xt

In [ ]:
execution.xt.sum()

In [ ]:
executionagent.test(1)

In [ ]:
execution.xt.sum()

In [ ]:
len(executionagent.memory)

<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="https://tpq.io" target="_blank">https://tpq.io</a> | <a href="https://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>